In [26]:
from __future__ import print_function
import sys
import cv2
import tracker
from random import randint
from video_read_thread import FileVideoStream


In [27]:
# Set video to load
videoPath = r"C:/Users/HP/Downloads/VID20200314183342.mp4"
 
# Create a video capture object to read videos
cap=FileVideoStream(videoPath).start() 
# Read first frame
success, frame = cap.read()
# quit if unable to read the video file
if not success:
    print('Failed to read video')
    sys.exit(1)

AttributeError: 'FileVideoStream' object has no attribute 'start'

In [3]:
## Select boxes
bboxes = []
colors = [] 
 
# OpenCV's selectROI function doesn't work for selecting multiple objects in Python
# So we will call this function in a loop till we are done selecting all objects
while True:
  # draw bounding boxes over objects
  # selectROI's default behaviour is to draw box starting from the center
  # when fromCenter is set to false, you can draw box starting from top left corner
    bbox = cv2.selectROI('MultiTracker', frame)
    bboxes.append(bbox)
    print("Box selected")
    colors.append((randint(0, 255), randint(0, 255), randint(0, 255)))
    print("Press q to quit selecting boxes and start tracking")
    print("Press any other key to select next object")
    k = cv2.waitKey(0) & 0xFF
    if (k == ord('q')):  # q is pressed
        break
    break
print('Selected bounding boxes {}'.format(bboxes))
cv2.destroyAllWindows()

Box selected
Press q to quit selecting boxes and start tracking
Press any other key to select next object
Selected bounding boxes [(456, 266, 345, 127)]


In [4]:
# Specify the tracker type
trackerType = "CSRT"   
 
# Create MultiTracker object
multiTracker = cv2.MultiTracker_create()
 
# Initialize MultiTracker 
for bbox in bboxes:
    multiTracker.add(tracker.createTrackerByName(trackerType), frame, bbox)

In [5]:
# Process video and track objects
while cap.
    success, frame = cap.read()
    if not success:
        break

# get updated location of objects in subsequent frames
    success, boxes = multiTracker.update(frame)

# draw tracked objects
    for i, newbox in enumerate(boxes):
        p1 = (int(newbox[0]), int(newbox[1]))
        p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
        cv2.rectangle(frame, p1, p2, colors[i], 2, 1)

# show frame
    cv2.imshow('MultiTracker', frame)

# quit on ESC button
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Esc pressed
        break
    cap.release()
    cv2.destroyAllWindows()

In [24]:
ex=FileVideoStream(videoPath).start()

AttributeError: 'FileVideoStream' object has no attribute 'start'